## Step 1:

This will work if this notebook is placed in this folder (EzproxyProj/data/ezproxylogs/) 


Following folder structure is needed : EzproxyProj/data/ezproxylogs/csv/offcampus/201712 

Add one month's worth of daily log files to the same directory that has this file in it. In the same directory, create a folder called 'csv' with matching subfolder (offcampus/201712) 
## Step 2: 
Run the code in the next box - click in box and press Shift+Enter. 


In [21]:
# function to count domain hits

# Code by Petrina Collingwood 
# Modified by Josephine Choi 

def domain_count(filename):
    
    import csv
    import re
    # create csv file from log file
    with open(filename,'r') as fh:
        with open('csv/' + filename + '.csv','w') as outfile:
            for line in fh:
                print(re.sub(r'\n|"','',line), file=outfile)
    import pandas as pd
    from urllib.parse import unquote
    # create dataframe from csv file skipping malformed lines
    df = pd.read_csv('csv/' + filename + '.csv',sep=' ', error_bad_lines=False, header=None, encoding='utf-8')
    # remove unnecessary columns
    df.drop(df.columns[[2,5,6,8,9]], axis=1, inplace=True)
    # name columns
    df.columns = ['ip', 'session_id', 'user_id', 'date_time', 'url', 'size']
    # formate date/time column
    df['date_time'] = df['date_time'].map(lambda x: x.lstrip('['))
    df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%b/%Y:%H:%M:%S')
    # remove lines where user is not logged in
    df = df[df.user_id != "-"]
    # decode urls
    def decode_url(url):
        decoded_url = unquote(url)
        return decoded_url
    df['url'] = df.url.apply(decode_url)
    # remove excess columns for domain
    df.drop(['ip','session_id','size'], axis=1, inplace=True)
    # remove ezp string from start of url
    df['url'] = df['url'].str.replace(r'^http://ezproxy\.lib\.ryerson\.ca/login/\?url=', '')# remove http etc
    df['url'] = df['url'].str.replace(r'^http://www\.|^https://www\.|^http://|^https://', '')
    # remove ezproxy string from start of url
    def parse_url(url):
        if (url.startswith("ezproxy.lib.ryerson.ca/login?url=")) and ("http" in url):
            location = url.find("http")
            return url[location:]
        elif (url.startswith("ezproxy.lib.ryerson.ca/login?url=")):
            return "-"
        else:
            return url
    df['url'] = df.url.apply(parse_url)
    # remove http etc
    df['url'] = df['url'].str.replace(r'^http://www\.|^https://www\.|^http://|^https://', '')
    # remove rows where ezproxy string is the only url
    df = df[df.url != "-"]
    # remove spaces introduced by unquoting
    df['url'] = df['url'].str.replace(r'\n', '')
    # remove everything after : or / or ?
    df['url'] = df['url'].str.replace(r'[:/?].*$', '')
    # remove .ezp.lib.unimelb.edu.au from urls
    df['url'] = df['url'].str.replace(r'ezproxy\.lib\.ryerson\.ca', '')
    df['url'] = df['url'].str.replace(r'ezproxy\.lib\.ryerson\.ca', '-')
    df = df[df.url != "-"]
    # create new column of domains
    def get_domain(url):
        regexp = re.compile(r'\.com|\.org|\.net|\.edu|-org|-com|\.gov')
        if regexp.search(url) is not None:
            for match in regexp.finditer(url):
                location = match.start()
            new_url = url[:location]
            if ('.' in new_url):
                location = new_url.rfind('.')
            elif ('-' in new_url):
                location = new_url.rfind('-')
            else:
                return url
            location += 1
            
            if ("-org" in url[location:]):
                modified_url = url[location:].replace(r'-org', '.org')
            elif ("-com" in url[location:]):
                modified_url = url[location:].replace(r'-com', '.com')
            else:
                return url[location:]
            return modified_url
        else:
            return url
    df['domain'] = df.url.apply(get_domain)
    # remove duplicate rows which have same user_id, date-time and domain. 
    df.drop_duplicates(subset=['user_id', 'date_time','domain'], inplace=True)
    df_domains = df['domain'].value_counts().reset_index()
    # rename columns
    df_domains.columns = ['domain', 'count']
    # create csv file from daily domain count dataframe
    df_domains.to_csv('daily_domains.csv',index=False, encoding='utf-8')

    


## Step 3: 
Run the code in the following box. This will run for up to an hour. 

Wait until the circle in the top right corner is no longer full before moving on to next step.


In [ ]:
# main code to process analyse daily log files and create domain count csv file
import glob
import csv
import pandas as pd
import os
# loop through files in current directory which begin with 'ezproxy.log.*'
# modify this if your log file name starts with something else
# JC: I have modified this so that it will look for the log files in a particular folder "/off/2012"
# JC: This also means that I do not have to delete the csv files later. However, I believe I do need to create separate folders under the "csv" folder 


for filename in glob.iglob(os.path.join("offcampus/201806/","*.log")):
    filename = filename.replace("\\","/")
    # call log analysis function 
    domain_count(filename)
    # create dataframe from csv output file from daily domain count
    df_daily = pd.read_csv('daily_domains.csv',sep=',', encoding='utf-8')
    import os.path
    # if domains.csv already exists - add daily domain counts to domains.csv
    if os.path.isfile('domains.csv'):
        # create dataframe from domains.csv
        df = pd.read_csv('domains.csv',sep=',')
        # rename daily dataframe count column to 'daily_count'
        df_daily.rename(columns={'count': 'daily_count'}, inplace=True)
        # merge the daily and accumulative dataframes
        df_merge = pd.merge(df, df_daily, on='domain', how='outer')
        # replace all NaN values with 0
        df_merge.fillna(0, inplace=True)
        # add new column with total of the two count columns
        df_merge['total_count'] = df_merge['count'] + df_merge['daily_count']
        # drop first 2 columns
        df_merge.drop(df_merge.columns[[1,2]], axis=1, inplace=True)
        # rename count column
        df_merge.rename(columns={'total_count': 'count'}, inplace=True)
        # create csv file from resulting dataframe
        df_merge.to_csv('domains.csv',index=False, encoding='utf-8')
    # first time through, convert domain count dataframe results to csv file
    else:
        df_daily.to_csv('domains.csv',index=False, encoding='utf-8')


## Step 4

Check that the domains.csv file looks ok. It should have 2 columns: domain and count.

Rename it to something like this: '2015_07_ezproxy_database_usage.csv'
Delete daily_domains.csv
Delete all the csv files in the csv directory.
Delete all of the log files that you have just analysed and copy and paste in the next month's worth of log
files to be analysed.

## Step 5:
Repeat steps 3 and 4 until you have analysed all of the months that you want to process.

## Step 6:

For each monthly file you created in the above process, you need to run the code in the next box. 
Each time you run it, you will need to modify the filename to the file that you are running.
So this code starts with June 2016 - '2016_06_ezproxy_database_usage.csv'. The first time you run the code,
you also need to change the column name in the 'else' section to the month name, in this case it's 'Jun-16'. 
For subsequent times you run it, change the month column name in the 'if' section.

In [56]:
# combines monthly files into one spreadsheet
import csv
import pandas as pd
# create dataframe from monthly domain count csv file
# each time you run this code, change the file name as you work through each month
# creates dataframe from monthly totals csv file
df_one = pd.read_csv('2018_06_ezproxy_database_usage_offcampus.csv',sep=',', encoding='utf-8')
import os.path
# second or more run through
# merge monthly file with totals file
if os.path.isfile('2017-2018_ezproxy_usage_monthly_totals_offcampus.csv'):
    # create dataframe from existing totals csv file
    df_two = pd.read_csv('2017-2018_ezproxy_usage_monthly_totals_offcampus.csv',sep=',', encoding='utf-8')
    #drop rows with domain = 0
    df_two = df_two[df_two.domain != "0"]
    # merge two dataframes
    df_merge = pd.merge(df_one, df_two, on='domain', how='outer')
    # remove NaN values with 0
    df_merge.fillna(0, inplace=True)
    # rename count column to month/year
    # change 'Jul-15' to whatever month file you are running
    df_merge.rename(columns={'count': 'Jun-18'}, inplace=True)
    df_merge = df_merge[df_merge.domain != "0"]
    # create csv from resulting dataframe
    df_merge.to_csv('2017-2018_ezproxy_usage_monthly_totals_offcampus.csv',index=False, encoding='utf-8')
# first run through
else:
    # rename count of first monthly count column, so replace 'Jun-16' with the month you are starting with
    df_one.rename(columns={'count': 'Jun-18'}, inplace=True)
    #drop rows with domain = 0
    df_one = df_one[df_one.domain != "0"]
    # convert dataframe to csv
    df_one.to_csv('2017-2018_ezproxy_usage_monthly_totals_offcampus.csv',index=False, encoding='utf-8')

## Step 7: 

Depending on how many months and which months you are processing, rename columns in the code below.
Run the code below and your csv spreadsheet is complete. You can then open it in Excel, make it prettier and
save it as a .xlsx file if you like.

In [48]:
# create 6-month and yearly total columns
import csv
import pandas as pd
df = pd.read_csv('2016-2017_ezproxy_usage_monthly_totals.csv',sep=',', encoding='utf-8')
# Modified (from biannual total to total per semester)
df['Sep-Dec_16_Total'] = df['Sep-16'] + df['Oct-16'] + df['Nov-16'] + df['Dec-16']
df['Jan-Apr_17_Total'] = df['Jan-17'] + df['Feb-17'] + df['Mar-17'] + df['Apr-17'] 
df['May-Aug_17_Total'] = df['May-17'] + df['Jun-17'] + df['Jul-17'] + df['Aug-17']
#df['Yearly_Total'] = df['Jul-Dec_13_Total'] + df['Jan-Jun_16_Total']
df['Yearly_Total'] = df['Sep-Dec_16_Total'] + df['Jan-Apr_17_Total'] + df['May-Aug_17_Total']
#sort on Yearly_Total
df.sort_values(by='Yearly_Total', ascending=0, inplace=True)
# create csv file from dataframe
df.to_csv('2016-2017_ezproxy_usage_monthly_totals.csv',index=False, encoding='utf-8')

KeyError: 'Sep-16'